# Inverse Reachability Repositioning Step by Step

## 1. Offline

### IRM Data

- Sample: ./config/robocare_right_irm.npy
  - This sample has 7899 points.
  - The supported z of target is {0.7, 0.75, 0.8}.

| Column Index | Name                     | Unit       | Remark           |
| ------------ | ------------------------ | ---------- | ---------------- |
| 0            | Mobile Base x            | meter      | For query output |
| 1            | Mobile Base y            | meter      | For query output |
| 2            | Target Object z (height) | meter      | For query input  |
| 3            | EEP x                    | meter      | For IK solver    |
| 4            | EEP y                    | meter      | For IK solver    |
| 5            | EEP z                    | meter      | For IK solver    |
| 6            | EE Roll                  | **DEGREE** |                  |
| 7            | EE Pitch                 | **DEGREE** |                  |
| 8            | EE Yaw                   | **DEGREE** | Cr               |
| 9            | Manipulability           | -          |                  |
| 10           | Joint_0 value            | radian     |                  |
| 11           | Joint_1 value            | radian     |                  |
| >=12         | Joint_2... values        | radian     |                  |


### Configuration

In [11]:
base_radius = 0.3
irm_path = "../config/robocare_right_irm.npy"

### Initialization

In [12]:
%matplotlib widget

import numpy as np
import irm

reposition = irm.InverseReachabilityMap(irm_path, base_radius, is_jupyter=True)

In [13]:
print(reposition)

< Inverse Reachability Solver >
Configuration:
    Base radius: 0.300000 m
    IRM data: ../config/robocare_right_irm.npy
       Shape: (7899, 18)



## 2. Online

### 2.1. Input

Known (from offline)
1. `irm`: Inverse reachability map of the robot arm
2. `base_radius`: The radius of the robot base (base is assumed to be circular)

---

Input

1. `Pt`: Position of the target object (relative to the robot’s current pose) => `(x, y)`
2. `Obs`: Area list of ground obstacles => `[CollisionModel, ...]`
3. `Cr`: Constraints on the approach angle (relative to the robot heading) (-pi ~ pi) => `(min, max)`
4. `Ct`: Constraints on the approach angle (relative to the target heading) (-pi ~ pi) => `(min, max)`

In [4]:
from collision import CollisionBox

Pt = (0.3, 0.1)
Obs = [CollisionBox((0.1, 0.0), np.radians(30.0), 0.5, 0.3)]
Cr = (np.radians(0), np.radians(30.1))  # min, max
Ct = (np.radians(-180), np.radians(180)) # min, max

### 2.2. Process

1. Cut the range of `Cr` from `irm` and set it to `Fcut`.
2. ~~[SKIP] Scan the maximum points for the radius and angle by each circle in `Fcut`.~~
3. Extract only the maximum as a `Fmax`.
4. Wipe the `Fmax` in the range of `Ct`. => `Fwiped`
5. Remove all obstacle areas from `Fwiped` with the offset of `base_radius`. => `Fclean`

In [5]:
reposition.calc(Pt, Obs, Cr, Ct, (0.05, 1.0, 0.02))

######################
# 1. Fcut
######################
Cut the range of `Cr` from `Fraw` and set it to `Fcut`.
Lower constraints on Cr: %f deg 0.0
Upper constraints on Cr: %f deg 30.100000000000005
Input Cr range in degree:  [-90.0, -75.0, -60.0, -45.0, -30.0, -15.0, 0.0, 15.0, 30.0, 45.0, 60.0, 75.0, 90.0]


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

######################
# 2. Fmax
######################
And extract only the maximum as a `Fmax`.
sq_sections.shape:  (48,)
Fmax.shape:  (27, 18)
When Ct == 0.0 deg


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

######################
# 3. Fwiped
######################
Wipe the `Fmax` in the range of `Ct`. => `Fwiped`
min Ct:  -180.0
max Ct:  180.0
Fwiped.shape:  (3551, 20)
Interval is  0.02  [m].


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

######################
# 4. Fclean
######################
Remove all obstacle areas from `Fwiped` with the offset of `Rsize`. => `Fclean`
Fclean.shape:  (3551, 20)
Fclean.shape:  (1566, 20)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### 2.3. Output

Candidate poses
(sorted in descending order of manipulability)

- `Cr`: Constraints on the approach angle (relative to the robot heading)
- `Ct`: Constraints on the approach angle (relative to the target heading)
- Chage of the robot heading `Theta = Ct - Cr`

| Column Index | Name                     | Unit       | Remark           |
| ------------ | ------------------------ | ---------- | ---------------- |
| 0            | Ct                       | **DEGREE** |                  |
| 1            | Cr                       | **DEGREE** | EE Yaw           |
| 2            | Mobile Base x            | meter      | For query output |
| 3            | Mobile Base y            | meter      | For query output |
| 4            | Manipulability           | -          |                  |
| 5            | Target Object z (height) | meter      | For query input  |
| 6            | EEP x                    | meter      | Based on base_footprint |
| 7            | EEP y                    | meter      | Based on base_footprint |
| 8            | EEP z                    | meter      | Based on base_footprint |
| 9            | EE Roll                  | **DEGREE** | Based on base_footprint |
| 10           | EE Pitch                 | **DEGREE** | Based on base_footprint |
| 11           | EE Yaw                   | **DEGREE** | Cr, Based on base_footprint |
| 12           | Joint_0 value            | radian     |                  |
| 13           | Joint_1 value            | radian     |                  |
| >=14         | Joint_2... values        | radian     |                  |


In [6]:
candidates = reposition.get_candidates(num=-1)
for c in candidates[:2]:
    reposition.candidate_inspector(c)

num:  27


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Candidate:
	   Base Pose2D: (x: 0.692 m, y: 0.227 m, theta: -155.147 deg)
	            Cr: 30.000 deg
	            Ct: -125.147 deg
	      Target Z: 0.700 m
	Manipulability: 0.10802
	  End-effecotr: (x: 0.315 m, y: -0.105 m, z: 0.700 m)
	                (r: -90.000 m, p: 90.000 m, y: 30.000 m) based on [base_footprint]
	   Joint (deg):  ['16.1', '-22.7', '-62.7', '76.2', '72.3', '70.5', '145.9', '25.5']
Candidate:
	   Base Pose2D: (x: 0.491 m, y: 0.465 m, theta: -110.650 deg)
	            Cr: 30.000 deg
	            Ct: -80.650 deg
	      Target Z: 0.700 m
	Manipulability: 0.10802
	  End-effecotr: (x: 0.315 m, y: -0.105 m, z: 0.700 m)
	                (r: -90.000 m, p: 90.000 m, y: 30.000 m) based on [base_footprint]
	   Joint (deg):  ['16.1', '-22.7', '-62.7', '76.2', '72.3', '70.5', '145.9', '25.5']


## 3. Client-side

Service Output

1. `num_candidates` (uint8): The number of candidates
2. `candidates` (geometry_msgs/Pose2D[]): x, y, heading of the robot
3. `approach_angles` (float64[])
4. `manipulabilities` (float64[])
5. `joint_angles` (sensor_msgs/JointState[]): radian

```     
|                  [top view]         X
|                                     ^
|                                     |
|                              Y <---(O)  (base_footprint)
|~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
|      [top view]                  [left view]            [backward view]
|    45     0    -45           90 <-----| (hand)            90(X, impossible)
|      .    ^    .                   .  |     ###             \|/
|        .  |  .               120 .   180    ###        0 ----+---- -180
| 90 <------|------> -90      ============| (ROBOT)            |
|                                 table   |                   -90
|        (1st Z)          =>           (Y)          =>      (2nd Z) = LWrist_Pitch - 90
|    approach angle              look-down angle          hand angle (Thumb direction)
```

---

### Candidates에서 Cost Function 사용

Parameters

1. `preferred_heading`

In [7]:
preferred_heading = np.radians(-60)


2 type of service

- input --> raw candidates
- input --> (raw candidates)+(cost func) --> one new position